In [8]:
!pip install ollama
!pip install newsapi-python
!pip install requests beautifulsoup4

In [9]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
    try:
        # 웹페이지 내용 요청
        response = requests.get(url)

        # 요청이 성공했는지 확인
        if response.status_code != 200:
            print(f"Failed to retrieve the web page. Status code: {response.status_code}")
            return ""

        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.content, 'html.parser')

        # 불필요한 스크립트 및 스타일 태그 제거
        for script_or_style in soup(['script', 'style']):
            script_or_style.decompose()

        # 모든 텍스트 추출
        text = soup.get_text(separator=' ')

        # 앞뒤 불필요한 공백 제거
        text = ' '.join(text.split())

        return text

    except requests.exceptions.RequestException as e:
        print(f"Error during requests to {url}: {str(e)}")
        return ""



In [31]:
import requests
import ollama

# 네이버 API 키 설정
client_id = 'xIpH54vcwURuvzz8t0vl'
client_secret = 'dfHi2SP3CQ'

# API 요청 URL
url = 'https://openapi.naver.com/v1/search/news.json'

# 카테고리 매핑 (네이버 뉴스 API는 카테고리를 지원하지 않으므로 검색어를 사용)
categories = {
    '경제': '경제',
    'IT': 'IT',
    '과학': '과학',
    '사회': '사회',
    '문화': '문화',
    '스포츠': '스포츠',
    '건강': '건강'
}

# 사용자로부터 카테고리 선택 받기
print("카테고리를 선택하세요: 경제, IT, 과학, 사회, 문화, 스포츠, 건강")
user_choice = input("선택한 카테고리: ")

# 유효한 카테고리인지 확인
if user_choice in categories:
    search_keyword = categories[user_choice]
else:
    print("유효하지 않은 카테고리입니다. 기본적으로 '경제' 카테고리를 선택합니다.")
    search_keyword = '경제'

# 파라미터 설정
params = {
    'query': search_keyword,  # 검색어
    'display': 5,            # 5개의 뉴스 가져오기
    'sort': 'date'           # 날짜 기준 정렬
}

# 헤더 설정
headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret
}

# API 요청
response = requests.get(url, headers=headers, params=params)

# 응답을 JSON 형태로 파싱
data = response.json()

# 뉴스 정보 출력
if 'items' in data:
    articles = data['items']
    for i, article in enumerate(articles, 1):
        title = article['title']
        description = article['description']
        link = article['link']
        pub_date = article['pubDate']
        
        extracted_text = extract_text_from_url(link)
        article=extracted_text[:3000]
        
        response = ollama.chat(
        model="llama3.1:8b",
        messages=[
            {
                "role": "user",
                "content": "Please analyze the text and summarize it around the important parts" + article ,
            },
        ],
        )
        summary1 = (response["message"]["content"])

        response = ollama.chat(
        model="llama3.1:8b",
        messages=[
            {
                "role": "user",
                "content": "Please translate it into Korean" + summary1,
            },
        ],
        )
        summary2 = (response["message"]["content"])
        
        print(f"뉴스 {i}:")
        print(f"제목: {title}")
        print(f"내용: "+summary2)
        print(f"URL: {link}")
        print(f"발행일: {pub_date}")
        print("-" * 80)
else:
    print("뉴스를 가져오는데 실패했습니다.")


카테고리를 선택하세요: 경제, IT, 과학, 사회, 문화, 스포츠, 건강


선택한 카테고리:  경제


뉴스 1:
제목: 장가계? 우루루 몰려다니는 건 싫어...
내용: 장가 계(일본) 여행 장가 계(장가계)는 영화 아바타의 배경입니다. 자연 경관이 우수하고 가족이 함께 방문하기 적합한 곳입니다.

그러나, 단체 관광을 싫어하는 젊은층들에게 맞춤형 현지투어를 제공하는 서비스가 등장했다. 이 서비스는 고객들이 직접 항공편을 예약하고 현지 여행사가 최적화된 일정을 설계합니다. 노쇼핑, 노옵션, 노팁 정책으로 합리적인 가격에 프리미엄 서비스를 제공하며 가족이나 소규모 그룹이 프라이빗한 여행을 즐길 수 있습니다.

이 서비스는 휴대폰에서 터치만 몇 번하면 간편하게 견적을 요청할 수 있으며 모바일로 일정표와 동선을 확인할 수 있습니다. 항공권, 숙박, 교통 등 모든 사항에 대한 상담과 맞춤형 견적서 제공으로 여행 계획 단계부터 구성원의 의견을 반영하며 만족도 높은 여행이 가능해진다.

장가 계는 가족과 함께 방문하기 적합한 곳입니다. 하지만, 젊은층들에게 맞춤형 현지투어를 제공하는 서비스가 등장했다. 이 서비스는 고객들이 직접 항공편을 예약하고 현지 여행사가 최적화된 일정을 설계합니다. 노쇼핑, 노옵션, 노팁 정책으로 합리적인 가격에 프리미엄 서비스를 제공하며 가족이나 소규모 그룹이 프라이빗한 여행을 즐길 수 있습니다.

이 서비스는 휴대폰에서 터치만 몇 번하면 간편하게 견적을 요청할 수 있으며 모바일로 일정표와 동선을 확인할 수 있습니다. 항공권, 숙박, 교통 등 모든 사항에 대한 상담과 맞춤형 견적서 제공으로 여행 계획 단계부터 구성원의 의견을 반영하며 만족도 높은 여행이 가능해진다.

장가 계는 자연 경관이 우수하고 가족이 함께 방문하기 적합한 곳입니다. 하지만, 젊은층들에게 맞춤형 현지투어를 제공하는 서비스가 등장했다. 이 서비스는 고객들이 직접 항공편을 예약하고 현지 여행사가 최적화된 일정을 설계합니다. 노쇼핑, 노옵션, 노팁 정책으로 합리적인 가격에 프리미엄 서비스를 제공하며 가족이나 소규모 그룹이 프라이빗한 여행을 즐길 수 있습니다.

이 서비스는 휴대폰에서

In [12]:
# 테스트 URL
url ="https://n.news.naver.com/mnews/article/016/0002355315?sid=101"  # 여기에 실제 뉴스 기사 URL을 입력하세요
extracted_text = extract_text_from_url(url)

# 추출된 텍스트 출력 (처음 3000자만 출력)
article=extracted_text[:3000]
print(article)

하반기 전통시장 소득공제율 40→80% 상향 본문 바로가기 NAVER 뉴스 연예 스포츠 날씨 프리미엄 검색 언론사별 정치 경제 사회 생활/문화 IT/과학 세계 랭킹 신문보기 오피니언 TV 팩트체크 알고리즘 안내 정정보도 모음 구독 헤럴드경제 언론사 구독되었습니다. 메인 뉴스판에서 주요뉴스를 볼 수 있습니다. 보러가기 헤럴드경제 언론사 구독 해지되었습니다. 하반기 전통시장 소득공제율 40→80% 상향 입력 2024.08.28. 오전 11:18 기사원문 양영경 기자 양영경 기자 구독 구독중 구독자 0 응원수 0 더보기 추천 쏠쏠정보 0 흥미진진 0 공감백배 0 분석탁월 0 후속강추 0 댓글 본문 요약봇 본문 요약봇 도움말 자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다. 닫기 텍스트 음성 변환 서비스 사용하기 성별 남성 여성 말하기 속도 느림 보통 빠름 이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다. 본문듣기 시작 닫기 글자 크기 변경하기 가 1단계 작게 가 2단계 보통 가 3단계 크게 가 4단계 아주크게 가 5단계 최대크게 SNS 보내기 인쇄하기 숙박쿠폰 50만장 풀고 3종 세제지원 추진 온누리상품권 구매한도·카드형 할인율 높여 최상목(가운데) 경제부총리 겸 기획재정부 장관이 28일 오전 서울 종로구 정부서울청사에서 열린 경제관계장관회의에서 모두발언을 하고 있다. 임세준 기자 정부가 회사 제공 명절선물에 부가가치세 비과세를 적용하고, 하반기 지출 증가분과 전통시장 지출에 대한 소득공제율을 상향하는 등 ‘세제지원 3종 세트’로 소비를 촉진한다. 추석 연휴를 계기로 국내 관광이 활성화될 수 있도록 숙박쿠폰 50만장을 배포하고, 휴가지원사업에 참여할 중소·중견기업 근로자 5만명을 추가로 모집한다. 정부는 28일 발표한 ‘추석 민생안정대책’에 이 같은 내용의 내수 회복 가속화 방안을 담았다. 고물가·고금리로 얼어붙은 내수 시장

In [ ]:
text = "하반기 전통시장 소득공제율 40→80% 상향 본문 바로가기 NAVER 뉴스 연예 스포츠 날씨 프리미엄 검색 언론사별 정치 경제 사회 생활/문화 IT/과학 세계 랭킹 신문보기 오피니언 TV 팩트체크 알고리즘 안내 정정보도 모음 구독 헤럴드경제 언론사 구독되었습니다. 메인 뉴스판에서 주요뉴스를 볼 수 있습니다. 보러가기 헤럴드경제 언론사 구독 해지되었습니다. 하반기 전통시장 소득공제율 40→80% 상향 입력 2024.08.28. 오전 11:18 기사원문 추천 쏠쏠정보 0 흥미진진 0 공감백배 0 분석탁월 0 후속강추 0 댓글 본문 요약봇 본문 요약봇 도움말 자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다. 닫기 텍스트 음성 변환 서비스 사용하기 성별 남성 여성 말하기 속도 느림 보통 빠름 이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다. 본문듣기 시작 닫기 글자 크기 변경하기 가 1단계 작게 가 2단계 보통 가 3단계 크게 가 4단계 아주크게 가 5단계 최대크게 SNS 보내기 인쇄하기 숙박쿠폰 50만장 풀고 3종 세제지원 추진 온누리상품권 구매한도·카드형 할인율 높여 최상목(가운데) 경제부총리 겸 기획재정부 장관이 28일 오전 서울 종로구 정부서울청사에서 열린 경제관계장관회의에서 모두발언을 하고 있다. 임세준 기자 정부가 회사 제공 명절선물에 부가가치세 비과세를 적용하고, 하반기 지출 증가분과 전통시장 지출에 대한 소득공제율을 상향하는 등 ‘세제지원 3종 세트’로 소비를 촉진한다. 추석 연휴를 계기로 국내 관광이 활성화될 수 있도록 숙박쿠폰 50만장을 배포하고, 휴가지원사업에 참여할 중소·중견기업 근로자 5만명을 추가로 모집한다. 정부는 28일 발표한 ‘추석 민생안정대책’에 이 같은 내용의 내수 회복 가속화 방안을 담았다. 고물가·고금리로 얼어붙은 내수 시장에 온기를 불어넣을 수 있도록 추석연휴와 연계한 국내 관광과 소비 활성화, 투자 촉진 등에 초점을 맞췄다. 정부는 우선 소비를 뒷받침하기 위해 세제지원 카드를 꺼내 들었다. 기업의 근로자 복리후생 재화에 대한 부가세 비과세 혜택을 늘린다. 기존에는 경조사(10만원)나 명절·생일·창립기념일 등(10만원)에 한해 1인당 연간 20만원 이하 재화로 비과세 한도를 적용했는데 여기에 명절(10만원)을 추가한다. 즉, 회사가 직원들에게 지급하는 명절선물에 별도의 세제 혜택을 주는 것이다. 올해 하반기 전통시장에서 쓴 금액에 대한 소득공제율을 한시적으로 80%로 올린다. 신용카드 등의 사용금액이 연간 총급여액의 25%를 초과하면 신용카드·전통시장·대중교통 지출액 등의 일정 부분을 근로소득에서 공제해주는데, 전통시장 지출에 대해서는 하반기 공제율을 40%에서 80%로 상향한다는 의미다. 하반기 카드사용액 증가분에 대한 소득공제율도 10%에서 20%로 두 배 올린다. 지난해 하반기보다 지출액이 5% 이상 늘어난 경우 초과분에 적용한다. 전통시장 소비 진작과 지역경제 활성화를 위해 온누리상품권 활용도 유도한다. 내달에만 한시적으로 개인의 온누리상품권 월 구매한도를 150만원에서 200만원으로 확대하고, 모바일·카드형의 할인율은 10%에서 15%로 올린다. 온누리상품권 사용 제한업종을 40종에서 28종으로 축소하고, 올해 안에 골목형 상점가 300곳을 지정하는 등 사용처를 확대한다. 아울러 기업이 온누리상품권을 구매해 업무 추진 목적으로 지출한 금액에 대해서는 업무추진비 손금산입 특례를 적용할 수 있도록 관련 법 개정을 추진한다. 청탁금지법상 허용되는 식사비 가액이 3만원에서 5만원으로 오른 데 맞춰 농축수산물 선물가액 한도 상향도 검토한다. 비수도권 지역에서 숙박할 경우 할인해주는 숙박쿠폰도 50만장 배포한다. 숙박비가 7만원 이상일 때 최대 3만원, 7만원 미만은 2만원 할인 혜택을 제공한다. 사용기한도 10월 13일에서 11월 말까지 연장하고 지자체를 통한 추가 할인 방안 등도 검토한다. 정부와 중소·중견기업이 근로자의 국내여행을 지원하는 ‘휴가지원사업’ 참여자를 5만명 추가로 모집해 연간 수혜 인원을 20만명까지 늘린다. 이 사업은 정부와 기업이 각 10만원, 근로자가 20만원을 분담하는 방식으로 추진된다. 추석연휴 기간에는 고속도로 통행료를 면제한다. 철도·선박 이용료 할인, 국내선 이용객 대상 공항 주차장 이용료 면제 등의 혜택도 제공한다. 내달 코리아 둘레길 전 구간 개통과 연휴기간 청와대·궁 야간개장 운영 등 관광지를 신규로 개방하고 국가유산·미술관·국립자연휴양림 등의 입장료를 면제한다. 각 지역 내에서 소규모 관광단지가 활성화될 수 있도록 지정조건(단일면적 5만㎡ 이상→동일 시·군 합산면적 5만㎡ 이상)을 완화하고, 중앙·지방 공동컨설팅 지원 체계도 구축한다. 또 방한 관광을 확대하기 위해 중국 등 동아시아 노선을 중심으로 하반기 국제 항공노선을 25% 이상 증편하고, 면세점 할인 행사도 개최한다. 정부는 경제 활력을 끌어올리기 위해 공공투자를 확대하고 지방재정 집행에도 속도를 낸다는 방침이다. 공공기관 당겨집행(3700억원)과 불용 최소화(2500억원), 발주·용역 계획 확대(8700억원) 등을 통해 하반기 1조5000억원을 추가로 투자한다. 올해 신축 매입임대 5만7000호 확보를 위해 최대 3조5000억원 수준의 추가 재정을 집행한다. 지방재정은 이·불용을 최소화해 연말까지 6조5000억원을 추가로 집행한다. 하반기 민간투자 집행은 전년보다 8000억원 확대해 연간 집행 규모를 5조7000억원에 맞춘다는 계획이다. 이는 최근 5년래 최대 수준이다. 입지·환경 규제 합리화와 투자 인프라 보강, 행정절차 지원 등을 담은 ‘3차 투자 활성화 대책’은 올해 하반기 중 발표한다. 이 밖에 비수도권의 준공 후 미분양 주택을 취득한 뒤 5년 이상 임대할 경우 5년간 발생한 양도소득금액의 50%를 과세 대상에서 제외하는 방안도 추진한다. 이날부터 내년 말까지 취득한 취득가액 6억원 이하, 국민주택 규모(85㎡) 이하 주택이 대상이다. 양영경 기자 양영경 y2k@heraldcorp.com Co"

In [7]:
print(text)

하반기 전통시장 소득공제율 40→80% 상향 본문 바로가기 NAVER 뉴스 연예 스포츠 날씨 프리미엄 검색 언론사별 정치 경제 사회 생활/문화 IT/과학 세계 랭킹 신문보기 오피니언 TV 팩트체크 알고리즘 안내 정정보도 모음 구독 헤럴드경제 언론사 구독되었습니다. 메인 뉴스판에서 주요뉴스를 볼 수 있습니다. 보러가기 헤럴드경제 언론사 구독 해지되었습니다. 하반기 전통시장 소득공제율 40→80% 상향 입력 2024.08.28. 오전 11:18 기사원문 추천 쏠쏠정보 0 흥미진진 0 공감백배 0 분석탁월 0 후속강추 0 댓글 본문 요약봇 본문 요약봇 도움말 자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다. 닫기 텍스트 음성 변환 서비스 사용하기 성별 남성 여성 말하기 속도 느림 보통 빠름 이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다. 본문듣기 시작 닫기 글자 크기 변경하기 가 1단계 작게 가 2단계 보통 가 3단계 크게 가 4단계 아주크게 가 5단계 최대크게 SNS 보내기 인쇄하기 숙박쿠폰 50만장 풀고 3종 세제지원 추진 온누리상품권 구매한도·카드형 할인율 높여 최상목(가운데) 경제부총리 겸 기획재정부 장관이 28일 오전 서울 종로구 정부서울청사에서 열린 경제관계장관회의에서 모두발언을 하고 있다. 임세준 기자 정부가 회사 제공 명절선물에 부가가치세 비과세를 적용하고, 하반기 지출 증가분과 전통시장 지출에 대한 소득공제율을 상향하는 등 ‘세제지원 3종 세트’로 소비를 촉진한다. 추석 연휴를 계기로 국내 관광이 활성화될 수 있도록 숙박쿠폰 50만장을 배포하고, 휴가지원사업에 참여할 중소·중견기업 근로자 5만명을 추가로 모집한다. 정부는 28일 발표한 ‘추석 민생안정대책’에 이 같은 내용의 내수 회복 가속화 방안을 담았다. 고물가·고금리로 얼어붙은 내수 시장에 온기를 불어넣을 수 있도록 추석연휴와 연계한 국내 관광과 소비 

In [14]:
import ollama

response = ollama.chat(
    model="llama3",
    messages=[
        {
            "role": "user",
            "content": "summary text " + article  ,
        },
    ],
)
summary = (response["message"]["content"])
print(summary)

KeyboardInterrupt: 

In [38]:
response = ollama.chat(
    model="llama3",
    messages=[
        {
            "role": "user",
            "content": "translate in Korean " + summary  ,
        },
    ],
)
print(response["message"]["content"])

Here is the translation:

2022년 2학기 경제활성화 및 중소기업 지원을 위해 정부는 다음과 같은 조치를 발표했습니다.

첫째로, 회사의 직원에게 휴가 선물 제공하는 기업에 대하여 가치เพิ่어 증세 세제(가치เพิ่어 세제) 허용 한도를 20,000,000원에서 30,000,000원으로 증가하겠습니다.

둘째로, 전통시장 업계를 지원하기 위해 정부는 수입 감액율을 40%에서 80%로 상승시켜 줄 것입니다. 이러한 조치는 전통시장 부문 소medium-sized 기업을 지원하는 목적입니다.

셋째로, 모바일 쇼핑 쿠폰에 대한 한도액을 150,000원에서 200,000원으로 증가시키고, 할인율도 10%에서 15%로 상승시켜 줄 것입니다.

넷째로, 중소기업의 '업무 관련 경비' 감액제도를 확대할 것입니다.

다섯째로, 직원들이 도서 지방에 휴가를 가는 경우 출장 소지권을 제공하는 한도액을 증가시키고, 할인율도 10%에서 15%로 상승시켜 줄 것입니다.

여섯째로, 정부는 도서 지방의 중소기업 지원을 위해 재정 지원 및 컨설팅 서비스를 제공할 새로운 프로그램을 시작할 것입니다.

일곱째로, 정부는 공공 투자 한도액을 3,000,000억원에서 4,000,000억원으로 증가시키고, 사가 투자 한도액도 5,000,000억원에서 6,000,000억원으로 증가시켜 줄 것입니다.

여덟째로, 정부는 도서 지방의 중소기업 지원을 위해 재정 지원 및 컨설팅 서비스를 제공할 새로운 프로그램을 시작할 것입니다.

전반적으로 이러한 조치는 2학기 경제활성화, 중소기업 지원, 및 경제 성장 촉진을 목적으로 있습니다.
